In [2]:
# Glass and Transparency Effects for Images in Google Colab
# Install required libraries
!pip install pillow opencv-python matplotlib numpy scipy

import cv2
import numpy as np
from PIL import Image, ImageFilter, ImageEnhance, ImageDraw
import matplotlib.pyplot as plt
from scipy import ndimage
import random

# Upload your image
from google.colab import files
print("Upload your image:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Load the image
original_img = Image.open(filename)
original_array = np.array(original_img)

print(f"Image loaded: {original_img.size}")

# Method 1: Glass Blur Effect (Frosted Glass)
def create_glass_effect(img, blur_strength=15, distortion=5):
    """
    Creates a frosted glass effect
    """
    # Convert to numpy array
    img_array = np.array(img)

    # Ensure blur strength is odd and positive
    if blur_strength % 2 == 0:
        blur_strength += 1
    if blur_strength < 1:
        blur_strength = 1

    # Apply Gaussian blur for glass effect
    blurred = cv2.GaussianBlur(img_array, (blur_strength, blur_strength), 0)

    # Add random distortion for glass texture
    rows, cols = img_array.shape[:2]

    # Create distortion map
    noise_x = np.random.randint(-distortion, distortion, (rows, cols)).astype(np.float32)
    noise_y = np.random.randint(-distortion, distortion, (rows, cols)).astype(np.float32)

    # Create coordinate grids
    map_x, map_y = np.meshgrid(np.arange(cols), np.arange(rows))
    map_x = map_x.astype(np.float32) + noise_x
    map_y = map_y.astype(np.float32) + noise_y

    # Apply distortion
    glass_effect = cv2.remap(blurred, map_x, map_y, cv2.INTER_LINEAR)

    return Image.fromarray(glass_effect)

# Apply glass effect with corrected parameters
glass_img = create_glass_effect(original_img, blur_strength=21, distortion=3)  # 21 is odd

# Method 2: Transparency Overlay Effects
def create_transparency_overlay(img, alpha=0.5, color=(255, 255, 255)):
    """
    Creates a transparent color overlay
    """
    # Convert to RGBA if not already
    if img.mode != 'RGBA':
        img = img.convert('RGBA')

    # Create overlay
    overlay = Image.new('RGBA', img.size, color + (int(255 * alpha),))

    # Blend with original
    result = Image.alpha_composite(img, overlay)
    return result

# Create different transparency overlays
transparent_blue = create_transparency_overlay(original_img, alpha=0.3, color=(0, 100, 255))
transparent_white = create_transparency_overlay(original_img, alpha=0.4, color=(255, 255, 255))
transparent_black = create_transparency_overlay(original_img, alpha=0.3, color=(0, 0, 0))

# Method 3: Glassmorphism Effect (Modern UI Style)
def create_glassmorphism(img, blur_radius=10, transparency=0.2, border_width=1):
    """
    Creates modern glassmorphism effect
    """
    # Blur the image
    blurred = img.filter(ImageFilter.GaussianBlur(radius=blur_radius))

    # Reduce opacity
    if blurred.mode != 'RGBA':
        blurred = blurred.convert('RGBA')

    # Apply transparency
    alpha = blurred.split()[-1]
    alpha = alpha.point(lambda p: int(p * (1 - transparency)))
    blurred.putalpha(alpha)

    # Add subtle border effect
    draw = ImageDraw.Draw(blurred)
    width, height = blurred.size

    # Draw border
    for i in range(border_width):
        draw.rectangle([i, i, width-1-i, height-1-i],
                      outline=(255, 255, 255, 100), fill=None)

    return blurred

# Create glassmorphism effect
glassmorphism_img = create_glassmorphism(original_img, blur_radius=15, transparency=0.3)

# Method 4: Water Glass Distortion Effect
def create_water_glass_effect(img, wave_amplitude=10, wave_frequency=0.01):
    """
    Creates water glass distortion effect
    """
    img_array = np.array(img)
    rows, cols = img_array.shape[:2]

    # Create wave distortion
    x_coords, y_coords = np.meshgrid(np.arange(cols), np.arange(rows))

    # Apply sine wave distortion
    x_distorted = x_coords + wave_amplitude * np.sin(wave_frequency * y_coords)
    y_distorted = y_coords + wave_amplitude * np.sin(wave_frequency * x_coords)

    # Ensure coordinates are within bounds
    x_distorted = np.clip(x_distorted, 0, cols-1).astype(np.float32)
    y_distorted = np.clip(y_distorted, 0, rows-1).astype(np.float32)

    # Apply distortion
    distorted = cv2.remap(img_array, x_distorted, y_distorted, cv2.INTER_LINEAR)

    return Image.fromarray(distorted)

# Create water glass effect
water_glass_img = create_water_glass_effect(original_img, wave_amplitude=15, wave_frequency=0.02)

# Method 5: Broken Glass Effect
def create_broken_glass_effect(img, num_cracks=20, crack_thickness=2):
    """
    Creates a broken glass effect with crack lines
    """
    img_copy = img.copy()
    draw = ImageDraw.Draw(img_copy)
    width, height = img.size

    # Create random crack lines
    for _ in range(num_cracks):
        # Random start and end points
        x1, y1 = random.randint(0, width), random.randint(0, height)
        x2, y2 = random.randint(0, width), random.randint(0, height)

        # Draw crack line
        draw.line([(x1, y1), (x2, y2)], fill=(255, 255, 255, 150), width=crack_thickness)

        # Add some branching cracks
        if random.random() > 0.5:
            mid_x, mid_y = (x1 + x2) // 2, (y1 + y2) // 2
            branch_x = mid_x + random.randint(-50, 50)
            branch_y = mid_y + random.randint(-50, 50)
            draw.line([(mid_x, mid_y), (branch_x, branch_y)],
                     fill=(200, 200, 200, 100), width=1)

    return img_copy

# Create broken glass effect
broken_glass_img = create_broken_glass_effect(original_img.convert('RGBA'), num_cracks=25)

# Method 6: Gradient Transparency Effect
def create_gradient_transparency(img, direction='vertical', start_alpha=1.0, end_alpha=0.0):
    """
    Creates gradient transparency effect
    """
    if img.mode != 'RGBA':
        img = img.convert('RGBA')

    width, height = img.size
    alpha_gradient = Image.new('L', (width, height))

    if direction == 'vertical':
        for y in range(height):
            alpha_value = int(255 * (start_alpha + (end_alpha - start_alpha) * y / height))
            for x in range(width):
                alpha_gradient.putpixel((x, y), alpha_value)
    else:  # horizontal
        for x in range(width):
            alpha_value = int(255 * (start_alpha + (end_alpha - start_alpha) * x / width))
            for y in range(height):
                alpha_gradient.putpixel((x, y), alpha_value)

    # Apply gradient alpha
    img.putalpha(alpha_gradient)
    return img

# Create gradient transparency effects
gradient_vertical = create_gradient_transparency(original_img, 'vertical', 1.0, 0.2)
gradient_horizontal = create_gradient_transparency(original_img, 'horizontal', 0.8, 0.1)

# Method 7: Bubble Glass Effect
def create_bubble_glass_effect(img, num_bubbles=50, max_bubble_size=20):
    """
    Creates bubble glass effect with circular distortions
    """
    img_array = np.array(img)
    height, width = img_array.shape[:2]

    # Create bubble distortions
    for _ in range(num_bubbles):
        # Random bubble position and size
        center_x = random.randint(0, width-1)
        center_y = random.randint(0, height-1)
        radius = random.randint(5, max_bubble_size)

        # Create circular mask
        y_coords, x_coords = np.ogrid[:height, :width]
        mask = (x_coords - center_x)**2 + (y_coords - center_y)**2 <= radius**2

        # Apply lens distortion effect
        if np.any(mask):
            # Get the region
            y_indices, x_indices = np.where(mask)

            # Apply magnification effect
            for i in range(len(y_indices)):
                y, x = y_indices[i], x_indices[i]

                # Calculate distance from center
                dist = np.sqrt((x - center_x)**2 + (y - center_y)**2)
                if dist > 0:
                    # Apply lens distortion
                    factor = 1.2  # Magnification factor
                    new_x = center_x + (x - center_x) / factor
                    new_y = center_y + (y - center_y) / factor

                    # Ensure new coordinates are within bounds
                    new_x = int(np.clip(new_x, 0, width-1))
                    new_y = int(np.clip(new_y, 0, height-1))

                    img_array[y, x] = img_array[new_y, new_x]

    return Image.fromarray(img_array)

# Create bubble glass effect
bubble_glass_img = create_bubble_glass_effect(original_img, num_bubbles=30, max_bubble_size=25)

# Method 8: Layered Transparency (Multiple Glass Sheets)
def create_layered_glass_effect(img, num_layers=3):
    """
    Creates effect of multiple transparent glass layers
    """
    result = img.copy()

    for layer in range(num_layers):
        # Create slightly offset and blurred copy
        offset_x = (layer + 1) * 3
        offset_y = (layer + 1) * 2

        # Create new image with offset
        layer_img = Image.new('RGBA',
                             (img.size[0] + offset_x, img.size[1] + offset_y),
                             (0, 0, 0, 0))
        layer_img.paste(img, (offset_x, offset_y))

        # Crop back to original size
        layer_img = layer_img.crop((offset_x, offset_y,
                                   img.size[0] + offset_x,
                                   img.size[1] + offset_y))

        # Apply blur and transparency
        layer_img = layer_img.filter(ImageFilter.GaussianBlur(radius=2 + layer))

        # Reduce opacity
        if layer_img.mode != 'RGBA':
            layer_img = layer_img.convert('RGBA')

        alpha = layer_img.split()[-1]
        alpha = alpha.point(lambda p: int(p * (0.6 - layer * 0.1)))
        layer_img.putalpha(alpha)

        # Blend with result
        if result.mode != 'RGBA':
            result = result.convert('RGBA')
        result = Image.alpha_composite(result, layer_img)

    return result

# Create layered glass effect
layered_glass_img = create_layered_glass_effect(original_img, num_layers=4)

# Display all effects in a grid
effects = [
    ("Original", original_img),
    ("Frosted Glass", glass_img),
    ("Blue Transparency", transparent_blue),
    ("Glassmorphism", glassmorphism_img),
    ("Water Glass", water_glass_img),
    ("Broken Glass", broken_glass_img),
    ("Gradient Fade", gradient_vertical),
    ("Bubble Glass", bubble_glass_img),
    ("Layered Glass", layered_glass_img)
]

# Create a 3x3 grid display
fig, axes = plt.subplots(3, 3, figsize=(18, 18))
axes = axes.flatten()

for i, (title, img) in enumerate(effects):
    axes[i].imshow(img)
    axes[i].set_title(title, fontsize=14, fontweight='bold')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Save all effects
for title, img in effects:
    filename_save = f"{title.lower().replace(' ', '_')}_effect.png"
    img.save(filename_save)
    print(f"✅ Saved: {filename_save}")

# Bonus: Interactive Effect Adjuster
def create_custom_glass_effect(img, blur=10, transparency=0.3, distortion=5, color_tint=(255, 255, 255)):
    """
    Customizable glass effect with adjustable parameters
    """
    # Ensure blur is odd and positive
    if blur % 2 == 0:
        blur += 1
    if blur < 1:
        blur = 1

    # Apply blur
    blurred = img.filter(ImageFilter.GaussianBlur(radius=blur//2))  # PIL uses radius, not kernel size

    # Add color tint
    if blurred.mode != 'RGBA':
        blurred = blurred.convert('RGBA')

    tint_overlay = Image.new('RGBA', blurred.size, color_tint + (int(255 * transparency),))
    tinted = Image.alpha_composite(blurred, tint_overlay)

    # Add distortion if requested
    if distortion > 0:
        img_array = np.array(tinted)
        rows, cols = img_array.shape[:2]

        noise_x = np.random.randint(-distortion, distortion, (rows, cols)).astype(np.float32)
        noise_y = np.random.randint(-distortion, distortion, (rows, cols)).astype(np.float32)

        map_x, map_y = np.meshgrid(np.arange(cols), np.arange(rows))
        map_x = map_x.astype(np.float32) + noise_x
        map_y = map_y.astype(np.float32) + noise_y

        distorted = cv2.remap(img_array, map_x, map_y, cv2.INTER_LINEAR)
        tinted = Image.fromarray(distorted)

    return tinted

# Example custom effects
print("\n🎨 Creating custom variations...")

custom_effects = [
    ("Soft Blue Glass", create_custom_glass_effect(original_img, blur=20, transparency=0.2, distortion=2, color_tint=(100, 150, 255))),
    ("Heavy Frost", create_custom_glass_effect(original_img, blur=25, transparency=0.4, distortion=8, color_tint=(255, 255, 255))),
    ("Tinted Green", create_custom_glass_effect(original_img, blur=15, transparency=0.3, distortion=3, color_tint=(100, 255, 150))),
]

# Display custom effects
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, (title, img) in enumerate(custom_effects):
    axes[i].imshow(img)
    axes[i].set_title(title, fontsize=12, fontweight='bold')
    axes[i].axis('off')

    # Save custom effects
    filename_save = f"custom_{title.lower().replace(' ', '_')}.png"
    img.save(filename_save)

plt.tight_layout()
plt.show()

print("\n✨ All glass and transparency effects created!")
print("🎯 Try adjusting the parameters in create_custom_glass_effect() for unique results!")
print("📁 All images saved to your Colab files")

# Pro tip: Combine effects
print("\n💡 Pro Tip: You can combine multiple effects!")
print("Example: combined = create_gradient_transparency(create_glass_effect(your_img))")

Output hidden; open in https://colab.research.google.com to view.